In [ ]:
import os, sys
sys.path.append("..")
from config.load_config import load_yaml_config, to_dict, recursive_namespace
import torch

### Generate dataset

In [4]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule

In [5]:
config = load_yaml_config("../config_files/config_folded_c_and_s.yaml")

In [6]:
dm = get_datamodule(config)

Retrieving synthetic population from cached file.


___

### `Encoder3DMesh`

In [7]:
from models.Model import Encoder3DMesh

In [8]:
from utils.helpers import get_coma_args

In [9]:
coma_args = get_coma_args(config, dm)
coma_args["phase_input"] = False

In [10]:
encoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_enc", 
    "cheb_polynomial_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "downsample_matrices",
    "adjacency_matrices",
    "activation_layers"
]

decoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_dec", 
    "cheb_polynomial_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "upsample_matrices",
    "adjacency_matrices",
    "activation_layers" 
]

enc_config = {k: v for k,v in coma_args.items() if k in encoder_args}
dec_config = {k: v for k,v in coma_args.items() if k in decoder_args}

In [11]:
enc_config["latent_dim_content"] = 45

In [12]:
# encoder_model = Encoder3DMesh(**enc_config)

In [13]:
# decoder_model = Decoder3DMesh(**dec_config)

In [14]:
dm.train_dataset[1]["s_t"].shape

(20, 1002, 3)

In [15]:
datapoint = next(iter(dm.train_dataloader()))
# x = datapoint[0][:,0,:,:]
# z = encoder_model(x[0])
# z

___

### `EncoderTemporalSequence`

In [16]:
from models.Model4D import EncoderTemporalSequence

In [17]:
cine_encoder = EncoderTemporalSequence(enc_config, z_aggr_function="DFT", n_timeframes=20)

In [18]:
s_t = datapoint["s_t"]

In [19]:
cine_encoder(s_t)["mu"]

tensor([[ 6.0417e-01,  6.5462e+00, -2.2564e+00, -4.4299e+00,  5.7031e+00,
         -2.5979e+00,  2.3315e+00, -2.1144e+00, -4.1857e+00, -3.7394e+00,
         -6.5174e+00, -3.2427e+00, -5.1645e-01, -2.4021e-01,  4.1102e-02,
          5.8865e-01,  3.6037e+00, -1.2641e+00, -3.8258e+00, -2.2058e+00,
          4.5126e+00,  8.9765e-01, -3.9191e+00, -2.5399e-01, -6.9890e+00,
         -4.7335e+00,  2.6103e-01,  3.1131e+00,  3.4867e+00,  6.6729e+00,
         -2.0969e+00, -6.0428e+00, -1.6038e+00, -3.6682e-01, -1.2137e+00,
         -2.0515e+00, -4.6690e+00,  3.5679e+00, -1.8419e+00, -4.5710e-02,
         -7.7243e-01,  3.6057e-02, -1.7142e+00, -2.0190e+00,  2.9820e+00],
        [ 2.9437e-01,  6.5118e+00, -2.6049e+00, -5.1559e+00,  5.1117e+00,
         -2.4343e+00,  2.5563e+00, -2.0216e+00, -5.2724e+00, -3.8803e+00,
         -5.7338e+00, -2.5635e+00, -8.8992e-02, -1.1492e+00, -7.6532e-02,
          1.8070e+00,  4.8577e+00, -1.2077e+00, -4.1816e+00, -2.6504e+00,
          4.3346e+00,  1.0350e-01, -4

In [20]:
from models.EncoderPLModule import CineComaEncoder

In [21]:
PLEncoder = CineComaEncoder(cine_encoder, config)

In [22]:
import pytorch_lightning as pl

In [23]:
trainer = pl.Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(PLEncoder, dm)

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name  | Type                    | Params
--------------------------------------------------
0 | model | EncoderTemporalSequence | 237 K 
--------------------------------------------------
237 K     Trainable params
0         Non-trainable params
237 K     Total params
0.949     Total estimated model params size (MB)


A Jupyter Widget

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


A Jupyter Widget

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'training_recon_loss'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

___

### `ContentDecoder`

In [ ]:
from models.Model import ContentDecoder

___

### `StyleDecoder`

In [ ]:
from models.Model import StyleDecoder

___

### `Coma4D_C_and_S`

In [ ]:
from models.Model import Coma4D_C_and_S